# Wavelength calibration

In [1]:
import numpy as np
import glob
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'nearest'


### Loading crispy's dependencies

In [2]:
import sys
codefolder = '../../../../crispy'
if codefolder not in sys.path: sys.path.append(codefolder)
from crispy.tools.initLogger import getLogger
log = getLogger('crispy')
from crispy.params import Params
codefolder = '../../../crispy'
par = Params(codefolder)

### Generate monochromatic wavelengths

In [3]:
# Suppose we create a folder in crispy/ReferenceFiles/ called wavecalR50_770_test
par.wavecalDir = par.prefix+'/wavecalR50_770/'
# construct monochromatic wavelengths, doesn't really matter which wavelengths
lamlist = np.arange(700,845,10)
print(lamlist)


[700 710 720 730 740 750 760 770 780 790 800 810 820 830 840]


In [4]:
# construct images for each wavelength
from crispy.IFS import createWavecalFiles
createWavecalFiles(par,lamlist)

crispy - INFO - Using PSFlet gaussian approximation
crispy - INFO - Assuming slices are evenly spread in wavelengths
[ 699.  701.]
crispy - INFO - Number of detector pixels per lenslet: 13.384615
crispy - INFO - Rebinning final detector. Image has dimensions 1024x1024
crispy - INFO - Done.
crispy - INFO - Performance: 4 seconds total
crispy - INFO - Writing data to ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_700.fits
crispy - INFO - Using PSFlet gaussian approximation
crispy - INFO - Assuming slices are evenly spread in wavelengths
[ 709.  711.]


crispy - INFO - Number of detector pixels per lenslet: 13.384615
crispy - INFO - Rebinning final detector. Image has dimensions 1024x1024
crispy - INFO - Done.
crispy - INFO - Performance: 4 seconds total
crispy - INFO - Writing data to ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_710.fits
crispy - INFO - Using PSFlet gaussian approximation
crispy - INFO - Assuming slices are evenly spread in wavelengths
[ 719.  721.]
crispy - INFO - Number of detector pixels per lenslet: 13.384615
crispy - INFO - Rebinning final detector. Image has dimensions 1024x1024
crispy - INFO - Done.
crispy - INFO - Performance: 4 seconds total
crispy - INFO - Writing data to ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_720.fits
crispy - INFO - Using PSFlet gaussian approximation
crispy - INFO - Assuming slices are evenly spread in wavelengths
[ 729.  731.]
crispy - INFO - Number of detector pixels per lenslet: 13.384615
crispy - INFO - Rebinning final detector. Image has dimensions 1024x102

### Loading wavelengths/filelists

In [5]:
#par.lamlist = np.loadtxt(par.wavecalDir + "lamsol.dat")[:, 0]
par.lamlist=lamlist
par.filelist = np.sort(glob.glob(par.wavecalDir + "det*.fits"))

### Run calibration. Choose parameters wisely.

In [6]:
from crispy.tools.wavecal import buildcalibrations
buildcalibrations(par,
                  order=3, # Order of polynomial map of the PSFLets (between 1 and 5)
                  upsample=10, # Upsampling factor of the PSFLets, needs to be at least 5
                  nsubarr=1, # How many zones on the detector have their own PSFLets? 
                  genwavelengthsol=True, # Compute wavelength at the center of all pixels (need to do this at least once)
                  makehiresPSFlets=True, # Compute hires PSFLets (usually False)
                  parallel=True) # Use all computer cores

crispy - INFO - Building calibration files, placing results in ../../../crispy/ReferenceFiles/wavecalR50_770_test/
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_700.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_700.fits
crispy - INFO - Initializing PSFlet location transformation coefficients
crispy - INFO - Performing initial optimization of PSFlet location transformation coefficients for frame ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_700.fits
crispy - INFO - Performing final optimization of PSFlet location transformation coefficients for frame ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_700.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_770_test/det_710.fits
crispy - INFO - Initializing transformation coefficients with previous values
crispy - INFO - Performing final optimization of PSFlet location transformation coeffi

In [7]:
from crispy.tools.reduction import calculateWaveList
lam_mid,lam_end = calculateWaveList(par)
print lam_mid,lam_end
print 725.99610254-712.8795633

crispy - INFO - Reduced cube will have 10 wavelength bins
[ 707.  721.  735.  749.  763.  777.  791.  805.  819.  833.] [ 700.  714.  728.  742.  756.  770.  784.  798.  812.  826.  840.]
13.11653924
